In [1]:
import random
import numpy as np

class Network(object):

    def __init__(self, sizes):
        """The list ``sizes`` contains the number of neurons in the
        respective layers of the network.  For example, if the list
        was [2, 3, 1] then it would be a three-layer network, with the
        first layer containing 2 neurons, the second layer 3 neurons,
        and the third layer 1 neuron.  The biases and weights for the
        network are initialized randomly, using a Gaussian
        distribution with mean 0, and variance 1.  Note that the first
        layer is assumed to be an input layer, and by convention we
        won't set any biases for those neurons, since biases are only
        ever used in computing the outputs from later layers."""
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        self.weights = [np.random.randn(y, x) for x, y in zip(sizes[:-1], sizes[1:])]

    def feedforward(self, a):
        """Return the output of the network if ``a`` is input."""
        for b, w in zip(self.biases, self.weights):
            a = sigmoid(np.dot(w, a)+b)
        return a

    def SGD(self, training_data, epochs, mini_batch_size, eta,
            test_data=None):
        """Train the neural network using mini-batch stochastic
        gradient descent.  The ``training_data`` is a list of tuples
        ``(x, y)`` representing the training inputs and the desired
        outputs.  The other non-optional parameters are
        self-explanatory.  If ``test_data`` is provided then the
        network will be evaluated against the test data after each
        epoch, and partial progress printed out.  This is useful for
        tracking progress, but slows things down substantially."""
        if test_data: n_test = len(test_data)
        n = len(training_data)
        for j in range(epochs):
            random.shuffle(training_data)
            mini_batches = [
                training_data[k:k+mini_batch_size]
                for k in range(0, n, mini_batch_size)]
            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, eta)
            if test_data:
                print ("Epoch {0}: {1} / {2}".format(j, self.evaluate(test_data), n_test))
            else:
                print ("Epoch {0} complete".format(j))

    def update_mini_batch(self, mini_batch, eta):
        """Update the network's weights and biases by applying
        gradient descent using backpropagation to a single mini batch.
        The ``mini_batch`` is a list of tuples ``(x, y)``, and ``eta``
        is the step size/learning rate."""
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        for x, y in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)            #invoke the backpropagation algorithm
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
        self.weights = [w-(eta/len(mini_batch))*nw
                        for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b-(eta/len(mini_batch))*nb
                       for b, nb in zip(self.biases, nabla_b)]

    def backprop(self, x, y):
        """Return a tuple ``(nabla_b, nabla_w)`` representing the
        gradient for the cost function C_x.  ``nabla_b`` and
        ``nabla_w`` are layer-by-layer lists of numpy arrays, similar
        to ``self.biases`` and ``self.weights``."""
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        # feedforward
        activation = x
        activations = [x]                                                 #list to store all the activations, layer by layer
        zs = []                                                           #list to store all the z vectors, layer by layer
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation)+b
            zs.append(z)
            activation = sigmoid(z)
            activations.append(activation)
        # backward pass
        delta = self.cost_derivative(activations[-1], y) * \
            sigmoid_prime(zs[-1])
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())
        # Note that the variable l in the loop below is used a little
        # differently to the notation in Chapter 2 of the book.  Here,
        # l = 1 means the last layer of neurons, l = 2 is the
        # second-last layer, and so on.  It's a renumbering of the
        # scheme in the book, used here to take advantage of the fact
        # that Python can use negative indices in lists.
        for l in range(2, self.num_layers):
            z = zs[-l]
            sp = sigmoid_prime(z)
            delta = np.dot(self.weights[-l+1].transpose(), delta) * sp
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())
        return (nabla_b, nabla_w)

    def evaluate(self, test_data):
        """Return the number of test inputs for which the neural
        network outputs the correct result. Note that the neural
        network's output is assumed to be the index of whichever
        neuron in the final layer has the highest activation."""
        test_results = [(np.argmax(self.feedforward(x)), y)
                        for (x, y) in test_data]
        return sum(int(x == y) for (x, y) in test_results)

    def cost_derivative(self, output_activations, y):
        """Return the vector of partial derivatives \partial C_x /
        \partial a for the output activations."""
        return (output_activations-y)

#### Miscellaneous functions
def sigmoid(z):
    """The sigmoid function."""
    return 1.0/(1.0+np.exp(-z))

def sigmoid_prime(z):
    """Derivative of the sigmoid function."""
    return sigmoid(z)*(1-sigmoid(z))


In [2]:
!git clone https://github.com/krajani21/handwriting-recognizer.git
%cd handwriting-recognizer

import mnist_loader
training_data, validation_data, test_data = mnist_loader.load_data_wrapper()


Cloning into 'handwriting-recognizer'...
remote: Enumerating objects: 22, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 22 (delta 9), reused 12 (delta 5), pack-reused 0 (from 0)
Receiving objects: 100% (22/22), 16.20 MiB | 12.41 MiB/s, done.
Resolving deltas: 100% (9/9), done.
/content/handwriting-recognizer


**First network with 30 neurons in 1 hidden layer**

In [7]:
net = Network([784, 30, 10])

We will use stochastic gradient descent (SGD) on the training dataset with 30 epochs, mini-batch size of 10, and step size = 3.0

In [8]:
net.SGD(training_data, 30, 10, 3.0, test_data=test_data)

Epoch 0: 9015 / 10000
Epoch 1: 9241 / 10000
Epoch 2: 9273 / 10000
Epoch 3: 9336 / 10000
Epoch 4: 9352 / 10000
Epoch 5: 9366 / 10000
Epoch 6: 9376 / 10000
Epoch 7: 9391 / 10000
Epoch 8: 9404 / 10000
Epoch 9: 9424 / 10000
Epoch 10: 9419 / 10000
Epoch 11: 9437 / 10000
Epoch 12: 9417 / 10000
Epoch 13: 9452 / 10000
Epoch 14: 9452 / 10000
Epoch 15: 9466 / 10000
Epoch 16: 9469 / 10000
Epoch 17: 9457 / 10000
Epoch 18: 9467 / 10000
Epoch 19: 9478 / 10000
Epoch 20: 9467 / 10000
Epoch 21: 9483 / 10000
Epoch 22: 9464 / 10000
Epoch 23: 9486 / 10000
Epoch 24: 9492 / 10000
Epoch 25: 9488 / 10000
Epoch 26: 9488 / 10000
Epoch 27: 9467 / 10000
Epoch 28: 9500 / 10000
Epoch 29: 9494 / 10000


**Second network with 100 hidden neurons in 1 hidden layer**

Increasing the number of neurons to 100 in the hidden layer increases the complexity, and hence should increase the accuracy of the output

In [5]:
net2 = Network([784, 100, 10])

In [6]:
training_data, validation_data, test_data = mnist_loader.load_data_wrapper()
net2.SGD(training_data, 30, 10, 3.0, test_data=test_data)

Epoch 0: 8315 / 10000
Epoch 1: 8518 / 10000
Epoch 2: 8563 / 10000
Epoch 3: 8632 / 10000
Epoch 4: 9476 / 10000
Epoch 5: 9517 / 10000
Epoch 6: 9508 / 10000
Epoch 7: 9549 / 10000
Epoch 8: 9553 / 10000
Epoch 9: 9533 / 10000
Epoch 10: 9583 / 10000
Epoch 11: 9580 / 10000
Epoch 12: 9571 / 10000
Epoch 13: 9595 / 10000
Epoch 14: 9620 / 10000
Epoch 15: 9612 / 10000
Epoch 16: 9620 / 10000
Epoch 17: 9627 / 10000
Epoch 18: 9606 / 10000
Epoch 19: 9629 / 10000
Epoch 20: 9637 / 10000
Epoch 21: 9618 / 10000
Epoch 22: 9642 / 10000
Epoch 23: 9654 / 10000
Epoch 24: 9667 / 10000
Epoch 25: 9661 / 10000
Epoch 26: 9657 / 10000
Epoch 27: 9662 / 10000
Epoch 28: 9653 / 10000
Epoch 29: 9656 / 10000


In the new network with 100 neurons in the hidden layer, just in epoch 5, we achieved an accuracy > 95%, which took the old network with 30 neurons, 30 epochs to achieve such an accuracy. Th accuracy also increased to 96.56% from 94.94% as expected, due to the increased complexity in the hidden layer